<a href="https://colab.research.google.com/github/rodrigoromanguzman/Actividades_Aprendizaje-/blob/main/language_model_rnn_from_zero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Utility libraries
import random
import re


from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
news_s_path = "/content/drive/MyDrive/newsSpace"

In [ ]:
# Utility function for detecting urls
def is_url(s):
    # A simple regex to check for a basic URL structure
    return re.match(r'https?://', s) is not None


In [ ]:
# We will separate the batches as the elements from an individual article
data_articles = []
# Lets collect all the unique ocurrences of each word
# to create the dictionary
vocabulary = set([])
num_articles = 5
counter = 0
with open(news_s_path, encoding='ISO-8859-1') as f:
    while (counter<num_articles):
      line = f.readline()
      if not line:
          break
      data_line = line.strip().split()
      url_index = next((i for i, item in enumerate(data_line) if is_url(item)), None)
      # We will get only the articles which have a url
      if url_index is not None:
        # We take after the url
        article = line.strip().split()[url_index+1::]
        [vocabulary.add(i) for i in article]
        data_articles.append(article)
      counter += 1
print("Vocabulary size -> ", len(vocabulary))
# print(data_articles[:100])
# print(list(vocabulary)[:200])


Vocabulary size ->  22


In [ ]:
def preprocess_data(data_articles, vocabulary):
    # Lowercase and tokenize the text
    processed_articles = []
    for article in data_articles:
        processed_article = [word.lower() for word in article if word.isalpha()]
        processed_articles.append(processed_article)
        vocabulary.update(processed_article)
    return processed_articles, vocabulary

data_articles, vocabulary = preprocess_data(data_articles, vocabulary)

In [ ]:
# Splitting data into training and test
test_percentage = 0.2
random.shuffle(data_articles)
split_point = int(len(data_articles)*test_percentage)
training_set = data_articles[split_point::]
test_set = data_articles[:split_point:]

In [ ]:
# Word embedding initialization with random values
mean = 0
# std_dev = 0.001
vocab_size = len(vocabulary)
embedding_size = 220 #Hardcoded value for the size of the vectors
word_embeddings = np.random.uniform(-0.5, 0.5, (vocab_size, embedding_size))
word_embeddings = word_embeddings / embedding_size  # Scaling by embedding size
hidden_size = 500  # Size of the hidden state vectors

# Create a dictionary to map words to their embedding vectors
word_to_embedding = {}
for i, word in enumerate(vocabulary):
    word_to_embedding[word] = word_embeddings[i]

In [ ]:
# Initialize weights randomly
# W_e = np.random.normal(mean, std_dev, size=(hidden_size, embedding_size))
# W_h = np.random.normal(mean, std_dev, size=(hidden_size,hidden_size))
# W_y = np.random.normal(mean, std_dev, size=(hidden_size,vocab_size))# Hidden to output
# bh = np.zeros((hidden_size, 1))  # Hidden bias
# by = np.zeros((vocab_size, 1))  # Output bias
# def initialize_parameters(vocab_size, embedding_size, hidden_size):
#     # Xavier initialization for weights
W_e = np.random.randn(hidden_size, embedding_size) * np.sqrt(2.0 / (hidden_size + embedding_size))
W_h = np.random.randn(hidden_size, hidden_size) * np.sqrt(2.0 / (hidden_size + hidden_size))
W_y = np.random.randn(hidden_size, vocab_size) * np.sqrt(2.0 / (vocab_size + hidden_size))

# Biases initialized to zero
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

#     return W_e, W_h, W_y, bh, by

In [ ]:
# One hot encoded vocabulary
vocabulary_list = list(vocabulary)

vocabulary_list.sort()  # Sorting to ensure consistent indexing

# Create a dictionary that maps words to indices based on the sorted order
word_to_index = {word: i for i, word in enumerate(vocabulary)}
word_to_index['<UNK>'] = len(vocabulary)  # Add '<UNK>' token with a new index

In [ ]:
def sentence_to_indices(sentence, word_to_index):
    return [word_to_index.get(word, word_to_index['<UNK>']) for word in sentence.split()]

In [ ]:
def softmax(x):
  return np.exp(x)/sum(np.exp(x))

In [ ]:
def tahnh(x):
  return np.tanh(x)

In [ ]:
# Input the probabilies and the true value
def cross_entropy(y_probs, y_true):
    # y_probs is a dictionary with timestep as key and array of probabilities as value
    # y_true is a list or array of true class indices
    total_loss = 0
    for t, true_index in enumerate(y_true):
        if t in y_probs:
            # Select the probability corresponding to the true class at timestep t
            prob_true_class = y_probs[t][true_index, 0]  # Adjust indexing if necessary
            # Calculate the negative log of this probability
            total_loss += -np.log(prob_true_class)
    # Average the loss over all timesteps
    average_loss = total_loss / len(y_true)
    return average_loss


In [ ]:
def apply_dropout(X, dropout_rate):
    mask = np.random.binomial(1, 1 - dropout_rate, size=X.shape)
    return np.multiply(X, mask)

In [ ]:
# Input the sentnece as thie index representation
# parameters: dictionary with keys -> W_e, W_h, W_y, bh, by
def forward_pass(indices, word_embeddings,parameters):
    h_prev = np.zeros((hidden_size, 1))  # Initial hidden state

    states = {'es': {},'hs': {},'ys': {},'ps': {}}
    states['hs'][-1] = np.copy(h_prev)


    # Forward pass
    for t in range(len(indices)):
        states['es'][t] = word_embeddings[indices[t]].reshape(-1, 1)  # Embedding vector for current input word
        # es_t_reshaped = states['es'][t].reshape(-1, 1)  # Reshape to (150, 1)

        # print("parameters['W_h']")
        # print(parameters['W_h'].shape)
        # print("h_prev")
        # print(h_prev.shape)
        # print("w_e")
        # print(parameters['W_e'].shape)
        # print("es_t_reshaped")
        # print(es_t_reshaped.shape)
        # print("parameters['bh']")
        # print(parameters['bh'].shape)

        h_rec = np.dot(parameters['W_h'], h_prev) + np.dot(parameters['W_e'], states['es'][t]) + parameters['bh']
        # print("HREC")
        # print(h_rec.shape)
        states['hs'][t] = tahnh(h_rec)  # Hidden state

        states['ys'][t] = np.dot(parameters['W_y'].T, states['hs'][t]) + parameters['by']  # Unnormalized log probabilities for next words
        states['ps'][t] = softmax(states['ys'][t])  # Probabilities for next words
        h_prev = states['hs'][t]  # Pass the current hidden state to the next time step
        # states['hs'][t] = apply_dropout(states['hs'][t], 0.2)
    # print("Hidden state:", states['hs'][t])
    # print("Predictions:", states['ps'][t])
    return states

In [ ]:
# Backpropagation
# outputs -> dictionary of dictionaries: es, hs, ys, ps
# parameters -> dictionary with: W_h, W_y, bh, by

def backward_pass(indices,outputs, parameters):
    gradients = {
        'dW_e': np.zeros_like(W_e),
        'dW_h': np.zeros_like(parameters['W_h']),
        'dW_y': np.zeros_like(parameters['W_y']),
        'dbh': np.zeros_like(parameters['bh']),
        'dby': np.zeros_like(parameters['by'])
    }

    dh_next = np.zeros_like(outputs['hs'][0])
    for t in reversed(range(len(indices)-1)):
        dy = np.copy(outputs['ps'][t])  # Copy softmax probabilities

        true_label_index = indices[t]
        dy[true_label_index] -= 1

        temp_product = np.dot(dy, outputs['hs'][t].T)  # Results in shape (1448, 500)
        gradients['dW_y'] += temp_product.T

        gradients['dby'] += dy
        dh = np.dot(parameters['W_y'], dy) + dh_next  # Backprop into h
        dh_rec = (1 - outputs['hs'][t] * outputs['hs'][t]) * dh  # Backprop through tanh nonlinearity
        gradients['dbh'] += dh_rec

        gradients['dW_h'] += np.dot(dh_rec, outputs['hs'][t-1].T)
        gradients['dW_e'] += np.dot(dh_rec, outputs['es'][t].T)
        dh_next = np.dot(parameters['W_h'].T, dh_rec)

    for gradientKey in gradients:
        np.clip(gradients[gradientKey], -5, 5, out=gradients[gradientKey])  # Clip to mitigate exploding gradients
    return gradients


In [ ]:
# Update weights
def update_parameters(parameters, gradients, learning_rate):
    # parameters and gradients are dictionaries with the same keys: 'W_e', 'W_h', 'W_y', 'bh', 'by'
    for key in parameters.keys():
        parameters[key] -= learning_rate * gradients['d' + key]

In [ ]:

# And we have our parameters 'W_e', 'W_h', 'W_y', 'bh', 'by'
parameters = {
    'W_e': W_e,
    'W_h': W_h,
    'W_y': W_y,
    'bh': bh,
    'by': by
}
def initialize_gradients(parameters):
    gradients = {}
    for key in parameters.keys():
        gradients['d' + key] = np.zeros_like(parameters[key])
    return gradients

# Usage
gradients = initialize_gradients(parameters)
learning_rate = 0.01 #0.005

In [ ]:
# Training
def training(mini_batches,gradients,parameters,vocabulary,learning_rate,epochs):
# def training(data_articles, word_to_index, word_embeddings, parameters, vocabulary_list, learning_rate, epochs):
    for epoch in range(epochs):
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0

        for batch_idx, sentence in enumerate(data_articles, start=0):
            # print("Sentence")
            # print(sentence)
            indices = [word_to_index.get(word, word_to_index['<UNK>']) for word in sentence]  # Tokenize the sentence

            # Forward pass
            outputs = forward_pass(indices, word_embeddings, parameters)

            # Create true labels (next words) for this batch
            true_labels = indices[1:]  # Assuming the next word is the true label for each word in the sentence

            # Backward pass
            gradients = backward_pass(indices, outputs, parameters)

            # Update parameters
            update_parameters(parameters, gradients, learning_rate)

            # Calculate and print loss
            loss = cross_entropy(outputs['ps'], true_labels)
            total_loss += loss


            # Calculate accuracy for this batch
            predicted_words = [vocabulary_list[np.argmax(outputs['ps'][output])] for output in outputs['ps']]
            # print("Predicted words")
            # print(predicted_words)
            predicted_indices = [word_to_index.get(word, word_to_index['<UNK>']) for word in predicted_words]
            correct_predictions += sum([1 for predicted, true in zip(predicted_indices[:-1], true_labels) if predicted == true])
            total_predictions += len(true_labels)

            # Print loss and accuracy at specified intervals

        # Print average loss and final training accuracy for the epoch
        if epoch % 5 == 0:  # Print some debug info every 10 epochs
          print("PREDICTED WORDS")
          print(predicted_words[:-1])
          print("Target words")
          print(sentence[1:])
          # print("How the probs look like")
          # print(outputs['ps'][0])
        avg_loss = total_loss / len(data_articles)
        final_accuracy = correct_predictions / total_predictions
        print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.4f}, Final Training Accuracy: {final_accuracy:.4f}")


In [ ]:
training(data_articles, gradients, parameters, vocabulary, learning_rate,5000)

PREDICTED WORDS
['Reuters', 'Reuters', 'Reuters', 'Into', 'Into', '(Reuters)', '(Reuters)', '(Reuters)']
Target words
['pullback', 'reflects', 'tech', 'blowout', 'none', 'business', 'reuters', 'wall']
Epoch 1, Average Loss: 3.5193, Final Training Accuracy: 0.0000
Epoch 2, Average Loss: 3.4419, Final Training Accuracy: 0.0000
Epoch 3, Average Loss: 3.2948, Final Training Accuracy: 0.0556
Epoch 4, Average Loss: 3.1736, Final Training Accuracy: 0.0556
Epoch 5, Average Loss: 3.1560, Final Training Accuracy: 0.0000
PREDICTED WORDS
['wall', 'Pullback', 'Pullback', 'Pullback', 'Pullback', 'Pullback', 'Pullback', 'Pullback']
Target words
['pullback', 'reflects', 'tech', 'blowout', 'none', 'business', 'reuters', 'wall']
Epoch 6, Average Loss: 3.2163, Final Training Accuracy: 0.0000
Epoch 7, Average Loss: 3.1438, Final Training Accuracy: 0.0000
Epoch 8, Average Loss: 3.2319, Final Training Accuracy: 0.0000
Epoch 9, Average Loss: 3.1317, Final Training Accuracy: 0.0000
Epoch 10, Average Loss: 3.2

KeyboardInterrupt: ignored